# Exercises 1.x

## Exercise 1.1
Compute the area of Italian geographic subdivisions (regions, provinces, municipalities) using GeoPandas and plot their area as a bar chart
- Download a shapefile that describe Italian regions (e.g., [here](https://gadm.org/download_country.html))
- Make a [bar chart](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.bar.html), put the regions in increasing order of area (put the region’s name on the x axis)
- Repeat for provinces and municipalities (plot only the 100 municipalities with the highest area)
- Plot the shape of each region (in blue), with the shape of its capital municipality (in red)


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import geopandas as gpd

### Regions

In [ ]:
regions = gpd.read_file('data/gadm41_ITA_shp/gadm41_ITA_1.shp', crs='EPSG:4326')
regions.set_index('NAME_1', inplace=True)
regions.head()

In [ ]:
regions['region_area'] = regions.area

In [ ]:
regions.head()

In [ ]:
regions.plot()

In [ ]:
regions.sort_values(by='region_area').plot(kind='bar')

### Provinces

In [ ]:
provinces = gpd.read_file('data/gadm41_ITA_shp/gadm41_ITA_2.shp', crs='EPSG:4326')
provinces.set_index('NAME_2', inplace=True)
provinces.head()

In [ ]:
provinces.plot()

In [ ]:
provinces['province_area'] = provinces.area

In [ ]:
fig = plt.figure(figsize=(20, 6))
ax = plt.axes()
provinces.sort_values(by='province_area').plot(kind='bar', ax=ax)

### Municipalities

In [ ]:
municipalities = gpd.read_file('data/gadm41_ITA_shp/gadm41_ITA_3.shp', crs='EPSG:4326')
municipalities.set_index('NAME_3', inplace=True)
municipalities.head()

In [ ]:
fig = plt.figure(figsize=(20, 20))
ax = plt.axes()
municipalities.plot(ax=ax)

In [ ]:
municipalities['mun_area'] = municipalities.area

In [ ]:
municipalities['mun_area'].sort_values()

In [ ]:
fig = plt.figure(figsize=(20, 6))
ax = plt.axes()
municipalities.sort_values(by='mun_area').tail(100).plot(kind='bar', ax=ax)

In [ ]:
fig = plt.figure(figsize=(20, 20))
ax = plt.axes()
ax_reg = regions.loc[['Lazio']].plot(color='k', ax=ax)
ax_prov = provinces.loc[['Roma']].plot(ax=ax_reg, color='b')
municipalities.loc[['Roma']].plot(ax=ax_prov, color='r')

In [ ]:
region_capitals = ["L' Aquila", 'Potenza', 'Catanzaro', 'Napoli', 'Bologna', 'Trieste', 'Roma', 'Genova',
                  'Milano', 'Ancona', 'Campobasso', 'Torino', 'Bari', 'Cagliari', 'Palermo', 'Firenze',
                  'Trento', 'Perugia', 'Aosta', 'Venezia']

In [ ]:
fig = plt.figure(figsize=(20, 20))
ax = plt.axes()
ax_reg = regions.plot(ax=ax, color='k')
ax_mun = municipalities[municipalities.index.isin(region_capitals)].plot(ax=ax_reg, color='r')        

## Exercise 1.2
Create and plot a GeoDataFrame with the top 1% and the bottom 1% municipalities in Italy based on their area:
- Download a shapefile that describe Italian regions (e.g., [here](https://gadm.org/download_country.html))
- Create a GeoDataFrame with two rows (top 1% and bottom 1%) and the corresponding multipolygons
- Plot the multipolygons with folium

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
import shapely

In [ ]:
municipalities = gpd.read_file('data/gadm41_ITA_shp/gadm41_ITA_3.shp', crs='EPSG:4326')
municipalities.set_index('NAME_3', inplace=True)
municipalities.head(2)

In [ ]:
municipalities['mun_area'] = municipalities.area

### Top 1%

In [ ]:
top_1_municipalities = municipalities.sort_values(by='mun_area', ascending=False).head(int(len(municipalities)*(1/100)))

In [ ]:
top_1_municipalities.plot()

### Bottom 1%

In [ ]:
bottom_1_municipalities = municipalities.sort_values(by='mun_area').head(int(len(municipalities)*(1/100)))

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.axes()
bottom_1_municipalities.plot(color='r', ax=ax)

In [ ]:
from shapely.geometry import MultiPoint, MultiLineString, MultiPolygon

In [ ]:
multi_poly_top_1 = MultiPolygon(list(top_1_municipalities['geometry']))

In [ ]:
multi_poly_top_1

In [ ]:
multi_poly_bottom_1 = MultiPolygon(list(bottom_1_municipalities['geometry']))

In [ ]:
multi_poly_bottom_1

In [ ]:
gdf = gpd.GeoDataFrame([['top1', multi_poly_top_1], ['bottom1', multi_poly_bottom_1]], columns=['cat', 'geometry'])
gdf

In [ ]:
import folium

In [ ]:
map_f = folium.Map(
    location=[multi_poly_bottom_1.centroid.y, multi_poly_bottom_1.centroid.x],
    tiles="cartodbpositron",
    zoom_start=5,
)

In [ ]:
for i, row in gdf.iterrows():
    sim_geo = gpd.GeoSeries(row['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange', 'color': 'blue', 'weight': 1})
    folium.Popup(row['cat']).add_to(geo_j)
    geo_j.add_to(map_f)
map_f

## Exercise 1.3
Several ["Los Pollos Hermanos"](https://en.wikipedia.org/wiki/Los_Pollos_Hermanos) vans, carrying large quantities of methamphetamine (meth), were attacked by a drug cartel 10 times in an area in New Mexico. The DEA thinks the meth lab is at the centroid of this area.
- Compute the smallest polygon that contains all the points corresponding to the attacks
- Create a `GeometryCollection` that contains New Mexico, the polygon, and the points within it
- Visualize the collection and the centroid in folium (use markers for points, color the centroid differently)
- Randomly generate the attacks’ points in New Mexico

In [ ]:
import geopandas as gpd
import osmnx as ox

In [ ]:
nm_gdf = ox.geocode_to_gdf('New Mexico, US')
ax = ox.project_gdf(nm_gdf).plot()
_ = ax.axis('off')

In [ ]:
nm_poly = nm_gdf['geometry'].iloc[0]
nm_poly

In [ ]:
minx, miny, maxx, maxy = nm_poly.bounds

In [ ]:
import numpy as np
from shapely.geometry import Point, MultiPoint
from shapely.geometry.collection import GeometryCollection

In [ ]:
n_points = 10
points = []
while len(points) < 10:
    pnt = Point(np.random.uniform(minx, maxx), np.random.uniform(miny, maxy))
    if nm_poly.contains(pnt):
        points.append(pnt)

In [ ]:
points = MultiPoint(points)
points

In [ ]:
ch_poly = points.convex_hull
ch_poly

In [ ]:
centroid = points.centroid

In [ ]:
collection = GeometryCollection([nm_poly, ch_poly, points])

In [ ]:
collection

In [ ]:
import folium

In [ ]:
map_f = folium.Map(location=[nm_poly.centroid.y, nm_poly.centroid.x], zoom_start=6)

In [ ]:
sim_geo = gpd.GeoSeries(collection)
geo_j = sim_geo.to_json()
geo_j = folium.GeoJson(data=geo_j,
            style_function=lambda x: {'fillColor': 'orange', 'color': 'blue', 'weight': 1})
geo_j.add_to(map_f)

folium.Marker(
    location=[centroid.y, centroid.x],
    popup="Meth Lab!",
    icon=folium.Icon(color="red", icon="info-sign"),
).add_to(map_f)

map_f